In [18]:
pip install pandas numpy scikit-learn nltk fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
import pandas as pd

# Télécharger les stop words français
import nltk
nltk.download('stopwords')
french_stop_words = stopwords.words('french')

# Exemple de données
data = {
    'title': ['Film A', 'Film B', 'Film C', 'Film D'],
    'description': [
        'Un film d\'action avec des explosions et des courses-poursuites.',
        'Une comédie romantique pleine de moments drôles et touchants.',
        'Un thriller psychologique avec des rebondissements inattendus.',
        'Un film d\'aventure avec des voyages et des découvertes.'
    ]
}


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zayani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Convertir en DataFrame
df = pd.DataFrame(data)
# **Créer la matrice TF-IDF et la similarité cosinus**
tfidf = TfidfVectorizer(stop_words=french_stop_words)
tfidf_matrix = tfidf.fit_transform(df['description'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 4️⃣ **Définir la fonction de recommandation**
def recommend_movies(title: str):
    try:
        if title not in df['title'].values:
            idx = np.random.randint(0, len(df))  # Choisir un film au hasard
        else:
            idx = df[df['title'] == title].index[0]

        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]

        movie_indices = [i[0] for i in sim_scores]
        return df['title'].iloc[movie_indices].tolist()

    except Exception:
        return df['title'].sample(3).tolist()  # Films aléatoires en cas d'erreur

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI

# Autoriser uvicorn à tourner dans Jupyter Notebook
nest_asyncio.apply()

app = FastAPI()

@app.get("/")
def home():
    return {"message": "API FastAPI fonctionne dans Jupyter Notebook !"}

# **Définir l'endpoint API**
@app.get("/recommend/")
def get_recommendations(title: str):
    recommendations = recommend_movies(title, cosine_sim, df)
    return {"title": title, "recommendations": recommendations}

# Lancer FastAPI directement dans Jupyter
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [30080]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:64840 - "GET /recommend/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:64840 - "GET /recommend/ HTTP/1.1" 422 Unprocessable Entity
